
# EDA - Index Funds and Macro Indicators
___________________________

Libraries

In [18]:
import pandas as pd

Data set

In [19]:
df= pd.read_excel('../df_SPY_VBMFX_Macro_Signals.xlsx',index_col=0,sheet_name='Sheet1')

In [20]:
df.head(3)

,SPY,SPY_mom,SPY_3m,SPY_3m_ann,SPY_3m_sd,SPY_3m_ann_sd,SPY_ln_3m,SPY_ann_ln_3m,VBMFX,VBMFX_mom,...,Slope_10Y_FF_Delta3m,AAA_minus_10Y,AAA_minus_10Y_Delta1m,AAA_minus_10Y_Delta3m,BAA_minus_10Y,BAA_minus_10Y_Delta1m,BAA_minus_10Y_Delta3m,BAA_minus_AAA,BAA_minus_AAA_Delta1m,BAA_minus_AAA_Delta3m
1993-04-30,24.479979,-0.025588,0.006866,0.027747,0.025016,0.050032,-0.025921,0.027369,2.488272,0.007439,...,-0.33,1.36,-0.24,-0.05,2.12,-0.06,0.01,0.76,0.18,0.06
1993-05-31,25.140205,0.026970,0.023106,0.095677,0.029115,0.058229,0.026613,0.091373,2.491764,0.001403,...,0.29,1.30,-0.06,-0.28,2.10,-0.02,-0.10,0.80,0.04,0.18
1993-06-30,25.230877,0.003607,0.004301,0.017316,0.026333,0.052666,0.003600,0.017168,2.536777,0.018065,...,-0.32,1.44,0.14,-0.16,2.16,0.06,-0.02,0.72,-0.08,0.14


In [21]:
df.tail(3)

,SPY,SPY_mom,SPY_3m,SPY_3m_ann,SPY_3m_sd,SPY_3m_ann_sd,SPY_ln_3m,SPY_ann_ln_3m,VBMFX,VBMFX_mom,...,Slope_10Y_FF_Delta3m,AAA_minus_10Y,AAA_minus_10Y_Delta1m,AAA_minus_10Y_Delta3m,BAA_minus_10Y,BAA_minus_10Y_Delta1m,BAA_minus_10Y_Delta3m,BAA_minus_AAA,BAA_minus_AAA_Delta1m,BAA_minus_AAA_Delta3m
2025-07-31,630.332458,0.023032,0.143197,0.707987,0.020495,0.040991,0.022770,0.535316,9.558923,-0.002986,...,0.20,1.04,-0.06,-0.17,1.67,-0.08,-0.29,0.63,-0.02,-0.12
2025-08-31,643.266602,0.020520,0.097672,0.451746,0.017142,0.034284,0.020312,0.372767,9.669253,0.011542,...,-0.18,1.19,0.15,0.09,1.80,0.13,-0.04,0.61,-0.02,-0.13
2025-09-30,666.179993,0.035620,0.081212,0.366607,0.008091,0.016183,0.035001,0.312331,9.770000,0.010419,...,0.16,1.06,-0.13,-0.04,1.67,-0.13,-0.08,0.61,0.00,-0.04


**Data set description**

The data set spans from **April 1993 to September 2025**. This length of **32.5** years gives us ample data to do a stock and bond regime analysis. We have 79 variables to be refined via feature selection.

>**Start Date**: 1993-04-30<br>
>**End Date**: 2025-09-30<br>
>**No. of Features**: 79<br>

In [22]:
data_shape = pd.DataFrame(
    {
        "n_rows": [df.shape[0]],
        "n_cols": [df.shape[1]],
        "start_date": [df.index.min() if isinstance(df.index, pd.DatetimeIndex) else None],
        "end_date": [df.index.max() if isinstance(df.index, pd.DatetimeIndex) else None],
    }
)
data_shape

,n_rows,n_cols,start_date,end_date
0,390,79,1993-04-30,2025-09-30
